<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dagngyen/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>
    h1 {
        padding: 8px 8px;
        background-image: linear-gradient(135deg, #c9f3ff, rgb(131, 218, 255));
        font-weight: 700;
        position: static;
        text-align: center;
        color: #006098;
        font-size: 36px;
    }
    h2 {
        font-weight: 700;
        text-align: center;
        font-style: italic;
        font-size: 24px;
    }
</style>

<div><h1>XÂY DỰNG MODEL</h1></div>
<div><h2>RAPIDS cuML SVR model</h2></div>

# Khai báo thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Đọc dữ liệu

In [2]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [3]:
# train = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/train.csv')
# train.sample(5)

In [4]:
# test = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/test.csv')
# test

In [5]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
train.sample(5)

,essay_id,full_text,score
10396,998557d,Started with the founding fathers. The Elector...,3
4424,41bd378,The challenge of exploring venus is very risky...,2
13455,c591447,I think theat limiting car usage in some citie...,3
3111,2e3312c,Have you evr wanted to help take care of anima...,3
16401,f1c81a1,"""The callenge of Exploring Venus,"" the author ...",2


In [6]:
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')
test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


# Xử lý dữ liệu thông qua trích xuất embeddings từ các mô hình Hugging Face

## Cài đặt thư viện cần thiết

In [7]:
# !pip install transformers torch cudf cuml

In [8]:
# !pip install transformers torch scikit-learn pandas

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch

models = [
    "bert-base-uncased",
    "roberta-base",
    "xlnet-base-cased",
    "gpt2",
    "distilbert-base-uncased",
    "albert-base-v2"
]

tokenizers = [AutoTokenizer.from_pretrained(model) for model in models]
models = [AutoModel.from_pretrained(model) for model in models]

# Thêm token padding cho các tokenizer không có nó
for tokenizer in tokenizers:
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/home/dagngyen/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dagngyen/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
essays = train['full_text'].to_numpy()  # Danh sách các bài luận cần xử lý
tokenized_essays = []

for tokenizer in tokenizers:
    tokenized_essay = [tokenizer(essay, padding='max_length', truncation=True, return_tensors='pt', max_length=1024) for essay in essays]
    tokenized_essays.append(tokenized_essay)


: 

In [ ]:
embeddings = []

for model, tokenized in zip(models, tokenized_essays):
    model_embeddings = []
    for essay in tokenized:
        with torch.no_grad():
            outputs = model(**essay)
            # Sử dụng output ẩn cuối cùng
            model_embeddings.append(outputs.last_hidden_state[:, 0, :].numpy())
    embeddings.append(np.array(model_embeddings))

# Nối tất cả các embeddings lại với nhau
combined_embeddings = np.concatenate(embeddings, axis=-1).reshape(len(essays), -1)